In [1]:
import sys
#Install packages prefer conda if you are using anaconda otherwise try with pip, uncomment only if install is needed (if you use Google colab pip should work)

#conda:
#os.system('conda install GPy -c conda-forge')
#os.system('conda install GPyOpt -c conda-forge')

#pip:
#!{sys.executable} -m pip install GPy
#!{sys.executable} -m pip install GPyOpt
import numpy as np
import matplotlib.pyplot as plt
import GPyOpt
from torchvision import datasets, transforms, utils
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ParameterSampler, RandomizedSearchCV, cross_val_score
from scipy.stats import uniform
import random

np.random.seed(32)
random.seed(32)

# Hyperparameters tuning using Bayesian Optimization with the GPyOpt library

As you are going to see, the project for this first part of the course will be to use Bayesian Optimization to tune the hyperparameters of a Deep Neural Network. To do that you are going to use a library that is more optimized compare to the code from scratch that we have written before. We are going to use [GPyOPt](https://sheffieldml.github.io/GPyOpt/) developed by the Machine Learning group of the University of Sheffield and it is based on GPy, which is a framework for using Gaussian Process in Python.

In this exercise you are going to use Bayesian Optimization to select the best hyperparameters of a random forest trained on part of the MNIST dataset. You are going also to compare it with respect to a random search on the hyperparameter space.


In [2]:
def load_MNIST():
    '''
    Function to load the MNIST training and test set with corresponding labels.

    :return: training_examples, training_labels, test_examples, test_labels
    '''

    # we want to flat the examples

    training_set = datasets.MNIST(root='./data', train=True, download=True, transform= None)
    test_set = datasets.MNIST(root='./data', train=False, download=True, transform= None)

    Xtrain = training_set.data.numpy().reshape(-1,28*28)
    Xtest = test_set.data.numpy().reshape(-1,28*28)

    ytrain = training_set.targets.numpy()
    ytest = test_set.targets.numpy()

    return Xtrain, ytrain, Xtest, ytest

In [3]:
## since training a random forest on the entire dataset takes some time
## we can consider only few labels, like 3, 5, 8 and 9

## we can load the training set and test set
Xtrain, ytrain, Xtest, ytest = load_MNIST()

## we use a mask to selects those subsets
#train_filter = np.isin(ytrain, [3, 5, 8, 9])
#test_filter = np.isin(ytest, [3, 5, 8, 9])

# apply the mask to the entire dataset
#Xtrain, ytrain = Xtrain[train_filter], ytrain[train_filter]
#Xtest, ytest = Xtest[test_filter], ytest[test_filter]

# print some information
print('Information about the new datasets')
print('Training set shape:', Xtrain.shape)
print('Test set shape', Xtest.shape)

100%|██████████| 9912422/9912422 [00:00<00:00, 33014780.56it/s]


Extracting ./data\MNIST\raw\train-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 28881/28881 [00:00<?, ?it/s]


Extracting ./data\MNIST\raw\train-labels-idx1-ubyte.gz to ./data\MNIST\raw



100%|██████████| 1648877/1648877 [00:00<00:00, 15976721.57it/s]


Extracting ./data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./data\MNIST\raw



100%|██████████| 4542/4542 [00:00<00:00, 4559724.45it/s]


Extracting ./data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./data\MNIST\raw

Information about the new datasets
Training set shape: (60000, 784)
Test set shape (10000, 784)


## Random forest

As you have seen in your Machine Learning course, there are a lot of hyperparameters to choose before training a Random Forest. If you look at the [sklearn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) you can notice all the paramters. For this exercise we will focus only on the following four hyperparameters:

1. **n_estimators**: the number of decision trees that are in the forest;
2. **criterion**: the criterion to evaluate the split. We should decide between *Gini impurity* (`gini`) and *information gain* (`entropy`);
3. **max_depth**: Maximum depth of the trees. If None, the tree expands until we have a element in all the leaves or  until all leaves contain less than min_samples_split samples.
4. **max_features**: The number of features to consider when looking for the best split. We have to choose among `'sqrt'` where we consider `sqrt(n_features)`, `'log2'` where we consider `log2(n_features)`. It is possible to use `None` where we consider all the features, but it makes everything super slow, so we are avoiding it.

Remember that when we are using Random Forest we can avoid running the cross-validation to get the validation error as approximation of the test error, but instead we can use the *out of bag* error to get an approximation of the test error we are going to get when we consider unseen example. 

<font color='blue'> Tasks:
1. You should define a `RandomForestClassifier` with default parameters, and compute the *out_of_bag* error (you should use `oob_score=True` when you define the classifier) and then the test error. This would be the baseline for the experiment.

In [5]:
model = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=32)
model.fit(Xtrain, ytrain)
print('Default model out of bag error', model.oob_score_)
print('Default model test accuracy', model.score(Xtest, ytest))

Default model out of bag error 0.9657833333333333
Default model test accuracy 0.9696


### Random search 

A possible way to find the best hyperparameters is to use random search over the parameter space. To perform this operation we can use `RandomizedSearchCV`. You can look at the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html). However, since we do not need cross-validation for Random Forests, we can use `ParameterSampler` to sample random parameters from our parameter space. Look at the documentation [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ParameterSampler.html#sklearn.model_selection.ParameterSampler) To be able to use this method you should first define a dictionary of the hyperparameters you want to optimize, in this case the four we mentioned above, and then decide a way to evaluate the model and how many folds for the cross-validation. The dictionary is of the form:

```python
params = {"name_params": uniform(0, 1), # if it is continuous between 0 and 1
           "name_params2": range(1,50), # if it is discrete but integer
           "name_params3": ['name1', 'name2']} # if it is discrete but string`
```

<font color='blue'> Tasks:

1. <font color='blue'> Crete the dictionary of the hyperparameters, we recommend keeping the value of the number of trees less than 100 otherwise it takes a very long time. You can try to increase it at home, but it will likely not be possible in the exercise session as you would be stuck for some time. After that, you can use the dictionary to create a random hyperparameter using `ParameterSampler`. The `ParameterSampler` call is already provided.

2. <font color='blue'> Using the parameters returned by the `ParameterSampler` you should fit a random forest, and for each iteration you should store the best value of the `model.oob_score_` you get. **HINT:** You can access the hyperparameters value by the name you used in the defined dictionary. (The whole process would take like 5/10 mins)

In [9]:
import time 

# hyperparams dictionary 
domain = {'n_estimators': [100, 200, 300, 400, 500],
          'max_depth': [None, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
          'min_samples_split': [2, 5, 10],
          'min_samples_leaf': [1, 2, 4],
          'max_features': ['auto', 'sqrt', 'log2', None],
          'bootstrap': [True, False]}
#rs = RandomizedSearchCV(model, param_distributions=domain, cv=3, verbose =2, n_iter=10)
#rs.fit(Xtrain, ytrain)

# create the ParameterSampler
param_list = list(ParameterSampler(domain, n_iter=20, random_state=32))
print('Param list')
print(param_list)
#rounded_list = [dict((k,v) for (k, v) in d.items()) for d in param_list]

#print('Random parameters we are going to consider')
#print(rounded_list)

## now we can train the random forest using these parameters tuple, and for
## each iteration we store the best value of the oob

current_best_oob = 0
iteration_best_oob = 0 
max_oob_per_iteration = []
i = 0
for params in param_list:
    print(i)
    print(params)
    model = RandomForestClassifier(**params, oob_score=True, random_state=32)

    start = time.time()
    model.fit(Xtrain, ytrain)
    end = time.time()
    model_oob = model.oob_score_
    print('OOB found:', model_oob)
    if model_oob > current_best_oob:
        current_best_oob = model_oob
        iteration_best_oob = i
    
    max_oob_per_iteration.append(current_best_oob)
    i += 1
    print(f'It took {end - start} seconds')
    

Param list
[{'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 'sqrt', 'max_depth': 40, 'bootstrap': False}, {'n_estimators': 500, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': None, 'max_depth': 80, 'bootstrap': True}, {'n_estimators': 500, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 70, 'bootstrap': True}, {'n_estimators': 100, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 50, 'bootstrap': False}, {'n_estimators': 200, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': None, 'max_depth': None, 'bootstrap': False}, {'n_estimators': 100, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_features': 'sqrt', 'max_depth': 80, 'bootstrap': False}, {'n_estimators': 300, 'min_samples_split': 10, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 10, 'bootstrap': True}, {'n_estimators': 400, 'min_samples_split': 5, 'min_samples_leaf': 4, 'max

ValueError: Out of bag estimation only available if bootstrap=True

### Bayesian Optimization

The procedure we are interested in, instead, is Bayesian Optimization. As before, also for GPyOpt, we should start by defining the set of hyperparameters you want to optimize. If some hyperparameters are discrete and represented by a string, you should integer instead and change them when you are initializing the classifier. The dictionary should have this form `{'name': 'name_of_params', 'type': 'type_of_variable', 'domain': domain_given_as_tuple}`. Note that the domain should specify as tuple. An example is given by

```python
params = [{'name': 'var_1', 'type': 'continuous', 'domain': (0,10)},
            {'name': 'var_2', 'type': 'discrete', 'domain': (0,5,10)},
            {'name': 'var_3', 'type': 'categorical', 'domain': (0,1)}]`
```

In the example above, suppose that `'var_3'` is usually used as string, where `0 = 'l1'` and `1 = 'l2'`, for example.
If you want to define a large discrete interval, you should define the tuple as `tuple(np.arange(1,100,1, dtype= np.int))`. 

The second ingredient we should define is the objective function we want to maximize. Since GPyOpt minimize things, when you are returning the value of the objective you return it negated. This function, usually defined as `objective_function(x)`, takes a general parameter `x`. This represents all the parameter we need. Therefore, when we define the function, we should collect the parameters by doing `param = x[0]`. Then we are able to pass the parameters to the classifier. In the example above we will have that `param[0]` is the 'var_1', `param[0]` is 'var_2', whereas, since 'var_3' is usually a string we should get back the string value, therefore `if param[2] == 0: var_3 = l1 else: var_3 = 'l2'`.
Note that by default GPyOpt initializes with 5 points and considers Bayesian optimization after this, therefore max_iter=15 will give you a total of 20 function evaluation. 
In a general way, the scheme for the objective function is similar to:

```python
def objective_function(x):

    param = x[0]
    
    if param[2] == 1:
        val_3 = 'l1'
    else:
        val_3 = 'l2'
        
    model = your_model(param[0],param[1],val_3)
    model.fit(X,y)
    
    ## then you will compute the measure you want to maximize
    ## usually you should consider a validation set
    ## in case of random forest you return the - oob score
    return - model.score(Xvalid, yvalid)
```

In case of random forest, you should return the `oob_score` and not the validation score. If you are going to use other models you should consider the validation set, instead.

When you have these two ingredients, you should create an Bayesian Optimization instancer using the following function `GPyOpt.methods.BayesianOptimization`, see the complete description [here](https://gpyopt.readthedocs.io/en/latest/GPyOpt.methods.html). Run it calling the `run_optimization` function of this instance, documentation [here](https://gpyopt.readthedocs.io/en/latest/GPyOpt.core.html).

At the end of the optimization, you can access the array of the best parameters by `x_best = opt.X[np.argmin(opt.Y)]`. You can also collect the best `oob_score` per iteration using 
`y_bo = np.maximum.accumulate(-opt.Y).ravel()`.


<font color='blue'> Tasks:

1. <font color='blue'>You should define all the ingredients needed for GPyOpt: 1) define the dictionary of hyperparameters, following the GPyOpt rules; 2) define the objective function you want to optimize. GPyOpt usually minimize (there is a parameter called maximize but it appears to have no effect), therefore, since in your setting you want to maximize the oob_score, you should return the negative oob_score; 3) run the optimization. When you create the `GPyOpt.methods.BayesianOptimization` instance you should define the following parameters: `f`,`domain`, `acquisition_type`, `exploration_weight`. The documentation for the GPyOpt package is quite limited and it is quite unclear to define how to define certain parameter, so elaborate hints are given here. The fitting procedure should take $\sim 10$ minutes. Do not worry about the warning "The set cost function is ignored! LCB acquisition does not make sense with cost." which might occur when using LCB. From discussions on github it appears to be a known error and might be fixed in the lastest git master branch.

2. <font color='blue'>In the same plot, show the best `oob_score` per iteration you obtain using the random search and the bayesian optimization. What do you see?

    

In [11]:
## define the domain of the considered parameters
n_estimators = tuple(np.arange(1,101,1, dtype= np.int))
# print(n_estimators)
max_depth = tuple(np.arange(10,110,10, dtype= np.int))
# max_features = ('log2', 'sqrt', None)
max_features = (0, 1)
# criterion = ('gini', 'entropy')
criterion = (0, 1)


# define the dictionary for GPyOpt
domain = [{'name': 'n_estimators', 'type': 'discrete', 'domain': n_estimators},
            {'name': 'max_depth', 'type': 'discrete', 'domain': max_depth},
            {'name': 'max_features', 'type': 'discrete', 'domain': max_features},
            {'name': 'criterion', 'type': 'discrete', 'domain': criterion}]


## we have to define the function we want to maximize --> validation accuracy, 
## note it should take a 2D ndarray but it is ok that it assumes only one point
## in this setting
def objective_function(x): 
    print(x)
    # we have to handle the categorical variables that is convert 0/1 to labels
    # log2/sqrt and gini/entropy

    #fit the model
    model = RandomForestClassifier(n_estimators=int(x[:,0]), max_depth=int(x[:,1]), max_features=['log2', 'sqrt', None][int(x[:,2])], criterion=['gini', 'entropy'][int(x[:,3])], oob_score=True, random_state=32)
    print(model.oob_score_)
    return - model.oob_score_


opt = GPyOpt.methods.BayesianOptimization(f = objective_function,   # function to optimize
                                              domain = domain,         # box-constrains of the problem
                                              acquisition_type ='MPI',       # Select acquisition function MPI, EI, LCB
                                                        # Select acquisition function MPI, EI, LCB
                                             )
opt.acquisition.exploration_weight=.1

opt.run_optimization(max_iter = 15) 

x_best = opt.X[np.argmin(opt.Y)]
print("The best parameters obtained: n_estimators=" + str(x_best[0]) + ", max_depth=" + str(x_best[1]) + ", max_features=" + str(
    x_best[2])  + ", criterion=" + str(
    x_best[3]))

AttributeError: module 'numpy' has no attribute 'int'.
`np.int` was a deprecated alias for the builtin `int`. To avoid this error in existing code, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
The aliases was originally deprecated in NumPy 1.20; for more details and guidance see the original release note at:
    https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations

In [ ]:
## comparison between random search and bayesian optimization
## we can plot the maximum oob per iteration of the sequence

# collect the maximum each iteration of BO, note that it is also provided by GPOpt in Y_Best
y_bo = 
# define iteration number
xs = 

plt.plot(xs, max_oob_per_iteration, 'o-', color = 'red', label='Random Search')
plt.plot(xs, y_bo, 'o-', color = 'blue', label='Bayesian Optimization')
plt.legend()
plt.xlabel('Iterations')
plt.ylabel('Out of bag error')
plt.title('Comparison between Random Search and Bayesian Optimization')
plt.show()

# [OPTIONAL] Investigate the acquisition function (not part of the exam)
<br>
<font color='blue'>
1. <font color='blue'> Investigate the dimensionality of the GP (look at the model subclass), how many dimensions are there? Why?
 
2.<font color='blue'> Take a look at the acqusition function, as this is more than 2 dimensional in this case it is not trivial to plot it, therefore keep the categorical variables fixed at some value that you decide and evaluate the acquisition function on a grid for the remaining two variables. The acquisition function can be evaluated using opt.acquisition.acquisition_function and you can create the grid using np.meshgrid. 